In [1]:
# coding: utf-8
"""
提案手法を実験するためのコード
"""
from filer2.filer2 import Filer
from library.graphsubmodular import GraphSubModular
from library.rouge import Rouge_N
import numpy as np
import glob

In [2]:
start = 40
end = 51

list_test_path = glob.glob('./files/OpinosisDataset1.0/preprocessed/test/*')
list_ans_path = glob.glob('./files/OpinosisDataset1.0/preprocessed/ans/*')

list_test_path.sort()
list_ans_path.sort()

list_test_path = list_test_path[start:]
list_ans_path = list_ans_path[start:]
print len(list_test_path)
print len(list_ans_path)

11
11


In [ ]:
directed = False

counter = 1

for test_path, ans_path in zip(list_test_path, list_ans_path):
    dict_test = Filer.readdump(test_path)
    dict_ans = Filer.readdump(ans_path)
    filename = ans_path.replace('./files/OpinosisDataset1.0/preprocessed/ans/', '')
    filename = filename.replace('.dump', '')
    print counter
    counter += 1
    # テスト用ファイルの作成
    # １００単語以上ある文は除く
    list_sentence = []
    list_sep = []
    list_sep_all = []
    list_edgelist = []
    for sen, sep, sepall, edge in zip(dict_test['sentence'],
                                      dict_test['sep_njv'],
                                      dict_test['sep_all'],
                                      dict_test['edge_njv']):
        if len(sepall) <= 100 and len(sep) != 0:
            list_sentence.append(sen)
            list_sep.append(sep)
            list_sep_all.append(sepall)
            list_edgelist.append(edge)

    list_edgelist = [[row1[0], row1[1]] for row in list_edgelist for row1 in row]
    # インスタンス化
    GM = GraphSubModular(list_sep_all=list_sep_all,
                         list_sep=list_sep,
                         list_edgelist=list_edgelist,
                         directed=directed,
                         inverse_flag=True)
    for scale in [0,1,2]:
        for r in [0.1, 0.5, 1, 2]:
            # 学習
            GM.m_greedy(num_w=100, r=r, scale=scale)
            # Rougeを計算するようにbag_of_wordsを作成する
            list_C = GM.list_C
            list_gm_test = [list_sentence[row[0]].replace(',', '').replace('.', '').lower().split(' ') for row in list_C]
            list_gm_ans = [[sen.replace(',', '').replace('.', '').lower().split(' ') for sen in row] for row in dict_ans['sentence']]
            try:
                R1 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=1)
            except:
                R1 = 0
            try:
                R2 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=2)
            except:
                R2 = 0
            try:
                R3 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=3)
            except:
                R3 = 0
            try:
                R4 = Rouge_N.rouge(list_gm_test, list_gm_ans, N=4)
            except:
                R4 = 0
            list_result = [[filename, scale, r, R1, R2, R3, R4]]
            Filer.writecsv(list_result, './files/OpinosisDataset1.0/result/GSM/njv_rev1/njv_undirected_%s_%s.csv'%(scale, r))